# Imports

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from itertools import product
from scipy.stats import laplace
    
df = pd.read_csv("movie_metadata.csv")
for i in df.index:
    if(df.isna().at[i,"gross"] or df.isna().at[i,"language"] or df.isna().at[i,"country"]): df = df.drop(i)

In [2]:
def topgross(df,eplison):
    if(df.shape[0] == 1): sensibilidade = 0 
    else : sensibilidade = np.max(df["gross"].values) - np.max(np.delete(df["gross"].values,np.where(df["gross"].values == [np.max(df["gross"].values)])))
    x = abs(laplace.rvs(loc = 0, scale = sensibilidade/eplison))
    return np.max(df["gross"].values) + x

# Score for the first Query

In [3]:
def score_gross(df,eplison):
    Dict = {}
    
    max_value = topgross(df,eplison)    
    #A = [df["movie_title"].values,df["movie_title"].values]
    #A = list(product(*A))
    
    #for mov1, mov2 in A:
    #    if(mov1 == mov2):
    #        value = df[df["movie_title"] == mov1]["gross"].values[0]
    #        Dict[mov1,mov2] = -abs(max_value - value)/max_value
    #    else: Dict[mov1,mov2] = 0
    
    for movie_title in df["movie_title"].values:
        value = df[df["movie_title"] == movie_title]["gross"].values[0]
        Dict[movie_title] = -abs(max_value - value)/max_value
        
    return Dict

In [4]:
def consulta1(df, eplison):
     
    fscore_df = score_gross(df,0.01*eplison) 
    sensibilidade = min(fscore_df.values())*(-1)
    
    total_sum = 0
    for movie_title in df["movie_title"].values:
        if(sensibilidade!=0): total_sum += np.e**(0.99*eplison*fscore_df[movie_title]/(2*sensibilidade))

    xk = df.index
    pk = []
    for idnx in df.index:
        if(sensibilidade != 0): pk.append(np.e**(0.99*eplison*fscore_df[df.at[idnx,"movie_title"]]/(2*sensibilidade))/total_sum)
        else: pk.append(1)
            
            
    custm = stats.rv_discrete(name='custm', values=(xk, pk))
    
    
    return df.at[custm.rvs(size=1)[0], "movie_title"],sensibilidade

In [5]:
def consulta2(df, eplison):
    result = []
        
    language_set = set(df["language"].values)
    for language in language_set:
        df_language = df[df["language"] == language]
        result.append(consulta1(df_language,eplison))
    
    return result

In [6]:
def topcount_country(df, eplison):
    group_by = df.groupby(["country"], as_index=False).count()
    if(df.shape[0] == 1): sensibilidade = 0
    else: sensibilidade = np.max(df["color"].values) - np.max(np.delete(df["color"].values,np.where(df["color"].values == [np.max(df["color"].values)])))
    x = abs(laplace.rvs(loc = 0, scale = sensibilidade/eplison))
    return np.max(df["color"].values) + x

# Score For the Third Query

In [7]:
def score_count(df, eplison):
    Dict = {}
    group_by = df.groupby(["country"], as_index=False).count()
    max_value = topcount_country(group_by, 0.01*eplison)
    
    for movie_country in df["country"].values:
        Dict[movie_country] = -abs(max_value-group_by[group_by["country"]== movie_country]["color"].values[0])/max_value
        
    return Dict

In [8]:
def consulta3_aux(df, eplison):
    
    fscore_df = score_count(df,eplison)  
    sensibilidade = min(fscore_df.values())*(-1)
    
    
    total_sum = 0
    for movie_country in df["country"].values:
        if(sensibilidade!=0): total_sum += np.e**(0.99*eplison*fscore_df[movie_country]/(2*sensibilidade))
    
   
    xk = df.index
    pk = []
    for idnx in df.index:
        if(sensibilidade != 0): pk.append(np.e**(0.99*eplison*fscore_df[df.at[idnx,"country"]]/(2*sensibilidade))/total_sum)
        else: pk.append(1)
            
    custm = stats.rv_discrete(name='custm', values=(xk, pk))
    
    
    return df.at[custm.rvs(size=1)[0], "country"],sensibilidade
    

In [9]:
def consulta3(df, eplison):
    result = []
    sens = []
    r1 = consulta3_aux(df, eplison/3)
    result.append(r1[0])
    sens.append(r1[1])
    r2 = consulta3_aux(df[df["country"]!=r1[0]], eplison/3)
    result.append(r2[0])
    sens.append(r2[1])
    r3 = consulta3_aux(df[df["country"]!=r1[0]][df[df["country"]!=r1[0]]["country"]!=r2[0]], eplison/3)
    result.append(r3[0])
    sens.append(r3[1])
    return result,sens

# Parameters

In [10]:
eplisons = [0.1/3,1/3,10/3]

# Consulta 1

In [11]:
c1_results = []
c1_sens = []

for eplison in eplisons:
    c1 = consulta1(df,eplison)
    c1_results.append(c1[0])
    c1_sens.append(c1[1])

In [12]:
c1[1]

0.9999999093929844

# Consulta 2

In [13]:
c2_results = []
c2_sens = []

for eplison in eplisons:
    c2 = consulta2(df,eplison)
    c2 =np.array(c2)
    c2_results.append(c2[:,0])
    c2_sens.append(c2[:,1])

# Consulta 3

In [14]:
c3_results = []
c3_sens = []

for eplison in eplisons:
    c3 = consulta3(df,eplison)
    c3 =np.array(c3)
    c3_results.append(c3[0])
    c3_sens.append(c3[1])

In [15]:
c3

array([['USA', 'UK', 'France'],
       ['0.9999858457774301', '0.9998776380594122', '0.9994019563443486']],
      dtype='<U18')

In [16]:
data = []
budgets = [0.1,1,10]
i = 0
for budget in budgets:
    data.append([budget,c1_results[i],c2_results[i],c3_results[i],c1_sens[i],c2_sens[i],c3_sens[i]])
    i += 1

In [17]:
results = pd.DataFrame(data, columns = [ 'budget', 'result q1', 'result q2', 'result q3', 
                                        'sens q1', 'sens q2', 'sens q3'])

In [18]:
results

,budget,result q1,result q2,result q3,sens q1,sens q2,sens q3
0,0.1,Cheaper by the Dozen 2,"[Baahubali: The Beginning , I Served the King ...","[USA, UK, France]",1.0,"[0.0, 0.0, 0.0, 0.9996444671757713, 0.0, 0.999...","[0.9999998728992576, 0.9999958036293238, 0.999..."
1,1.0,Shanghai Surprise,"[Baahubali: The Beginning , I Served the King ...","[USA, France, Spain]",1.0,"[0.0, 0.0, 0.0, 0.9899785345534254, 0.0, 0.999...","[0.9999997772446698, 0.9999938204932263, 0.999..."
2,10.0,The Rose,"[Baahubali: The Beginning , I Served the King ...","[USA, UK, France]",1.0,"[0.0, 0.0, 0.0, 0.9698317260156591, 0.0, 0.999...","[0.9999858457774301, 0.9998776380594122, 0.999..."


In [19]:
results.to_csv('result.csv',index=False,sep=',')